In [ ]:
# del session.logger
# del session

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=31,
    use_group_norm=True,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=800,  # 5% total steps
    adalora_tfinal=12800,  # 50-80% total steps
    adalora_deltaT=400,  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=16000,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-3,
    epochs=40,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.8,
        "mse_loss": 0.2,
        "commitment_loss": 0.0,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 3e-4
config.batch_size = 128

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/objectives/CLIP_MSE_BATCH_LEVEL_128_3e-4",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() // 2 - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SimpleConv initialized with 9295984 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 8858112
Found 24 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
openai/whisper-tiny.en loaded with 8540472 frozen params (4 layers and 384 dim).
AdaLora has 332064 trainable params.


2025-02-05 00:24:49,128	INFO worker.py:1841 -- Started a local Ray instance.
Training Epoch 1:  93%|█████████▎| 126/135 [10:59<00:33,  3.71s/it]

Starting rank reallocation at step 800.


Training Epoch 1: 100%|██████████| 135/135 [11:40<00:00,  5.19s/it]


Testing completed in 0.95m.
Epoch 1 completed in 12.67m. 0.09m per recording.
New highest average test accuracy: 0.0122, lowest FinLayer total loss: 1.2576 at epoch 1.


...

Testing completed in 0.97m.
Epoch 25 completed in 12.56m. 0.09m per recording.


Training Epoch 26: 100%|██████████| 135/135 [11:38<00:00,  5.17s/it]


Testing completed in 0.99m.
Epoch 26 completed in 12.63m. 0.09m per recording.


Training Epoch 27: 100%|██████████| 135/135 [11:35<00:00,  5.15s/it]


Testing completed in 0.97m.
Epoch 27 completed in 12.57m. 0.09m per recording.
Early stopping at epoch 27. Highest top 10 accuracy at epoch 6.
Training completed.
unseen_subject: Acc: 0.1315, Top 5: 0.3540, Top 10: 0.4840
Loss: 4.1355, Mel Loss: 3.1351, Clip Loss: 3.7615, MSE Loss: 0.6299, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 3.3719, FinLayer MSE Loss: 0.5907
FinLayer CosSim Loss: 0.2243, FinLayer Total Loss: 1.0004
unseen_task: Acc: 0.0907, Top 5: 0.3125, Top 10: 0.4743
Loss: 4.1943, Mel Loss: 3.2121, Clip Loss: 3.8608, MSE Loss: 0.6170, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 3.2700, FinLayer MSE Loss: 0.5953
FinLayer CosSim Loss: 0.2253, FinLayer Total Loss: 0.9823
unseen_both: Acc: 0.1138, Top 5: 0.3250, Top 10: 0.4725
Loss: 4.2099, Mel Loss: 3.2162, Clip Loss: 3.8700, MSE Loss: 0.6008, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 3.3444, FinLayer MSE Loss: 0.5886
FinLayer CosSim Loss: 0.2236, FinLayer Total Loss: 0.9938


In [ ]:
# del session.logger
# del session

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=31,
    use_group_norm=True,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=800,  # 5% total steps
    adalora_tfinal=12800,  # 50-80% total steps
    adalora_deltaT=400,  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=16000,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-3,
    epochs=40,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.8,
        "mse_loss": 0.2,
        "commitment_loss": 0.0,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 1e-4
config.batch_size = 128

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/objectives/CLIP_MSE_BATCH_LEVEL_128_1e-4",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() // 2 - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SimpleConv initialized with 9295984 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 8858112
Found 24 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
openai/whisper-tiny.en loaded with 8540472 frozen params (4 layers and 384 dim).
AdaLora has 332064 trainable params.


2025-02-05 09:52:31,700	INFO worker.py:1841 -- Started a local Ray instance.
Training Epoch 1:  93%|█████████▎| 126/135 [10:42<00:32,  3.60s/it]

Starting rank reallocation at step 800.


Training Epoch 1: 100%|██████████| 135/135 [11:24<00:00,  5.07s/it]


Testing completed in 0.91m.
Epoch 1 completed in 12.35m. 0.09m per recording.
New highest average test accuracy: 0.0253, lowest FinLayer total loss: 1.1504 at epoch 1.


...

Training Epoch 14: 100%|██████████| 135/135 [11:18<00:00,  5.03s/it]


Testing completed in 0.96m.
Epoch 14 completed in 12.27m. 0.09m per recording.


Training Epoch 15: 100%|██████████| 135/135 [11:22<00:00,  5.05s/it]


Testing completed in 0.94m.
Epoch 15 completed in 12.31m. 0.09m per recording.


Training Epoch 16:  46%|████▌     | 62/135 [05:50<06:59,  5.75s/it] 

In [ ]:
# for name, param in session.model.encoder.named_parameters():
#     if "lora_A" in name or "lora_B" in name or "lora_E" in name:
#         print(name)

In [ ]:
# import time

# dataloader = session.get_dataloader(buffer_size=30, num_workers=24, max_cache_size=400)
# dataloader.start_fetching(session.dataset["train"], cache=True)

# # Process batches as they become available
# try:
#     batches, recs, start_time = 0, 0, time.time()
#     print(f"Total recordings: {len(session.dataset['train'])}")

#     while True:
#         batch = dataloader.get_recording()

#         if batch is None:
#             break

#         brain = batch.brain_segments["all"]
#         batches += brain.shape[0]
#         recs += 1

#         if recs % 10 == 0:
#             print(f'Processed {recs} recordings of {len(session.dataset["train"])}')

#     print(
#         f"Batch {batches} ({recs} recordings) processed in {time.time() - start_time:.2f}s"
#     )
#     print(
#         f"Average processing time per recording: {(time.time() - start_time) / recs:.2f}s"
#     )
#     print(
#         f"Average processing time per batch: {(time.time() - start_time) / batches:.2f}s"
#     )

# except KeyboardInterrupt:
#     print("Interrupted")
#     dataloader.stop()
# except Exception as e:
#     print("Error", e)
#     dataloader.stop()